# _Bengali.AI Handwritten Grapheme Classification_

En el siguiente trabajo se aborda la resolución del problema planteado para la clasificación de los componentes de grafemas benagalíes. 

Para más información [ingrese aquí](https://www.kaggle.com/c/bengaliai-cv19)

## 1. Importar módulos y cargar _dataset_

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2 as cv
import numpy as np
import pandas as pd 
from glob import glob
from tensorflow import keras
from keras.layers import Conv2D
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

En la lista podemos ver los archivos que conforman el _dataset_ propuesto para este problema. En nuestro caso sólo utilizaremos el archivo `train_image_data_0.parquet` y el archivo `train.csv` para el entrenamiento.

In [ ]:
train_image_0_df = pd.read_parquet(f'../input/bengaliai-cv19/train_image_data_0.parquet')

In [ ]:
train_tabular_df = pd.read_csv('../input/bengaliai-cv19/train.csv')

## 2. Análisis exploratorio

Para conseguir un entendimiento del problema, primero vamos a avanzar analizando las generalidades de los _dataframes_ que se cargaron en memoria. 

### 2.1. Información de los _dataframes_

In [ ]:
train_image_0_df.info()

In [ ]:
train_tabular_df.info()

In [ ]:
train_image_0_df.shape

In [ ]:
train_tabular_df.shape

En los comandos anteriores es posible observar que `train_image_0_df` posee 50210 imágenes. Entre sus columnas se observa el `image_id` y los píxeles de cada una de las imágenes.

Si restamos de las 32333 columnas el `image_id`, sólo restan las columnas que se corresponden a los 137x236 píxeles de cada imagen, tal como se informa en la seción [data](https://www.kaggle.com/c/bengaliai-cv19/data) de la documentación.

In [ ]:
train_image_0_df.head()

In [ ]:
train_tabular_df.head()

### 2.2. Valores únicos por columna


In [ ]:
train_tabular_df.nunique()

Cabe destacar la cantidad de valores únicos para cada una de las columnas. Pues esto definirá el tipo y cantidad de salidas de la red neuronal.

## 3. Ingeniería de _features_

### 3.1. Fusión de los _dataframes_

Es necesario contar con un único conjunto de datos, hasta ahora se tienen las imágenes y las etiquetas en tablas diferentes. Además, debido a que no se están utilizando todas las imágenes (por falta de memoria), es necesario descartar las etiquetas que no perteneces a una imagen dentro de `train_image_0_df`

In [ ]:
train_data =  pd.merge(train_image_0_df, train_tabular_df, on='image_id').drop(['image_id'], axis=1)

In [ ]:
train_data.shape

### 3.2. Separar las variables independientes (_features_) de las salidas (_labels_)

In [ ]:
train_labels = train_data[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme']]
train_labels.shape

In [ ]:
train_data = train_data.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme'], axis=1)
train_data.shape

### 3.3. Redimensionar las imágenes

Lamentablemente, es necesario redimiensionar las imágenes para que sea posible procesarlas con el _hardware_ disponible. Además, esto acelera el proceso de ensayo. Es probable que, una vez definido el modelo, valga la pena hace pruebas con imágenes más grandes.

In [ ]:
def resize(df, size=64, need_progress_bar=True):
    resized = {}
    for i in range(df.shape[0]):
        image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size))
        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

Al momento de llamar el método `resize()`, también se divide por 255 para obtener valores decimales en cada píxel. Esto permite que, durante el entrenamiento, se interprete correctamente la naturaleza del número como indicador de la intensidad de cada píxel.

In [ ]:
train_data = resize(train_data, size=64)/255
train_data = train_data.values.reshape(-1, 64, 64, 1)

## 4. Modelo

Debido a que los grafemas están compuestos por 3 componentes, se decidió entrenar 3 modelos con la misma estructura. El entrenamiento de cada uno de los modelos está enfocado a cada uno de las componentes del grafema.

La única diferencia que hay entre los modelos es la capa de salida. Si bien todos terminan con una activación _softmax_, cada uno lo hace con la cantidad de salidas correspondientes al número de valores únicos que tiene cada componente. 

In [ ]:
model_dict = {
    'grapheme_root': Sequential(),
    'vowel_diacritic': Sequential(),
    'consonant_diacritic': Sequential()
}
for model_type, model in model_dict.items():
    model.add(Conv2D(input_shape=(64,64,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(layers.BatchNormalization(momentum=0.15))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(128, 3, activation="relu", padding="same"))
    model.add(Conv2D(128, 3, activation="relu", padding="same"))
    model.add(MaxPool2D(2))
    model.add(Conv2D(256, 3, activation="relu", padding="same"))
    model.add(Conv2D(256, 3, activation="relu", padding="same"))
    model.add(MaxPool2D(2))
    model.add(Flatten())
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    if model_type == 'grapheme_root':
        model.add(layers.Dense(168, activation='softmax', name='root_out'))
    elif model_type == 'vowel_diacritic':
        model.add(layers.Dense(11, activation='softmax', name='vowel_out'))
    elif model_type == 'consonant_diacritic':
        model.add(layers.Dense(7, activation='softmax', name='consonant_out'))
    model.compile(optimizer="adam", loss=['categorical_crossentropy'], metrics=['accuracy'])
    
plot_model(model_dict['grapheme_root'])

### 4.1. Entrenamiento de los modelos

Iterando sobre el diccionario de modelos es posible realizar el entrenamiento de los 3 modelos para detectar las componentes de cada grafema.

Cabe señalar algunos aspectos importantes del entrenamiento:

1. Se utiliza un `ImageDataGenerator` para hacer _data augmentation_.
1. Antes de entrenar a cada modelo se divide el _dataset_ en _train_ y _test_, de forma aleatoria, dejando el 10% de las filas para _test_.
1. El histórico de la evolución de las métricas se guarda en una lista.

In [ ]:
batch_size = 32
epochs = 500
history_list = []
model_types = ['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']
for target in model_types:
    Y_train = train_labels[target]
    Y_train = pd.get_dummies(Y_train).values
    x_train, x_test, y_train, y_test = train_test_split(train_data, Y_train, test_size=0.1, random_state=123)
    datagen = ImageDataGenerator()
    datagen.fit(x_train)
    history = model_dict[target].fit(datagen.flow(x_train, y_train, batch_size=batch_size), 
                                               epochs = epochs, validation_data = (x_test, y_test))
    history_list.append(history)

In [ ]:
model.save('./submission.csv')

### 4.2. Visualización del avance de las métricas en entrenamiento

In [ ]:
for history in history_list:
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

## 5. Conclusiones

Después de hacer repetidas pruebas, se concluye que:

1. El modelo da mejores resultados cuando se entrena para decidir entre menos categorías. Para el caso de los 168 _grapheme roots_, el modelo tiene su peor _performance_.
1. La capa `BatchNormalization` tiene un gran impacto en el _accuracy_.
1. VGG16 no llega a los resultados obtenidos con esta red.
1. 64x64 parece ser un tamaño razonable para las imágenes. Las pruebas con 96x96 y 128x128 resultaron en memoria insuficiente.
1. Trabajar en colab no fue posible por la cantidad de memoria necesaria.
1. Trabajar en _hardware_ local (sin GPU) resulta en entrenamientos de casi 3 horas, lo que dificulta el desarrollo.